# Treino do Yolo

### Preparações
Para baixar nosso dataset:

In [ ]:
%pip install roboflow

Em https://universe.roboflow.com/roboflow-jvuqo/football-players-detection-3zvbc recebemos o código abaixo, com nossa chave de api, selecionando o dataset e o modelo yolo que desejarmos.
Rodando esse código baixamos o dataset na pasta onde está esse notebook.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="6jaRGHIPR2qq1dtJQwJ7")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(12)
dataset = version.download("yolov8")

Em data.yaml vemos as definições de treino e dados da fonte do dataset. Inclusive as classes para descoberta. E fuçando nos arquivos do teste vemos as labels, que contém os dados referentes a cada imagem, com marcações que significam a classe em questão e coordenadas para sua boundingbox (o objeto encontrado, ou ensinado para ser encontrado), no formato: Xcenter Ycenter Width Height, relativo a cada imagem


Agora, precisamos justar os caminhos das pastas de treino no data.yaml. Isso por causa dos últimos requerimentos da biblioteca ultralytics.
Por simplicidade, podemos usar no terminal os comandos:

In [ ]:
!sed -i 's|\(train: \).*|\1../train/images|' training/football-players-detection-12/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' training/football-players-detection-12/data.yaml

Depois disso, precisamos deixar nossos datasets dentro de uma pasta "datasets". Se pa uma requisição do ultralytics. Não tenho ctz se precisa mesmo, mas deixa mais organizado. Atentar pra pegar o caminho inteiro até o data.yaml que vamos usar. -> right click>copy path. E também se quisermos pegar um modelo qualquer por ai.

## O Treino

A resolução de 1280 é, segundo Piotr melhor do que 640 para nossa detecção, por causa da bola. O problema é que isso aumenta bastante o tempo de treino e detecções.

"batch size is the number of samples processed in one foward and backward pass of training algorithm.". Epochs é algo como "quantas vezes passamos pelo dataset no treino".

Piotr usa epochs=50, batch=6, com o dataset completo. E creio que ele usa o modelo mais pesado do yolo. Impossível pra meu pc.

Rodar esse comando de treino a seguir irá criar, em runs>train>weights, os modelos de treino. E na própria pasta do notebook, também cria dois modelos, creio que são caso não indiquemos o modelo base a ser treinado, ai ele baixa.

Smaller dataset:
    Tentando fazer o treino com número bem reduzido de imagens. Apenas 5 imagens de cada pasta. A de treino foi a mais reduzida, de longe. 
    Com 2 batch`s conseguimos o uso mais otimizado das memórias da gpu em meu note.
    Qual o melhor treino que conseguiria fazer em uma noite?
    De qualquer forma, não tem jeito, vou ter que usar o modelo já treinado né?


### Bateria de testes de treino
dos modelos em meu note (NVIDIA GeForce GTX 1650, com 4096MiB de memória)

Ao longo de uma longa bateria de testes, vi que quanto maior o batch, maior a chance de um modelo não conseguir ser treinado. Creio que isso seja em função da potência da GPU usada.

- sd=smaller dataset
- fd=dataset completo (full dataset)
- loss=apresenta loss, nan=apresenta nan
- class=quando da nan, mas apresenta classes ao final

Modelo N
- sd, B2E3, 640, 24s, loss (sem classe?)
- sd, b2e1, 640, 22s, loss, sem classe
- sd, B2E10, 640, 38s, loss
- sd, b3e1, 1280,, nan
- sd, b3e3, 1280, 31s, nan
- sd, b2e1, 1280, 27s, nan
- sd, b1e3, 1280, 25s, nan
- sd, b3e3, 640, 26s, loss, sem classe
- sd, b4e3, 640, 27s, nan
- sd, b5e3, 640, 27s, nan
- N funciona com b3 para 640, não funciona com 1280.
Modelo M
- sd, b1e1, 54s 53s
- sd, b2e1, 57s 49s
- sd, b3e1, 55s
- fd, b1e1, 13m22s
- fd, b2e1, 12m52s
- fd, b3e1, 15m51s
- fd, b2e5, 58m18
- fd, b2e10, 117m
- sd, b2e10, +-60m.
- sd, B2E3, 640, 42s, nan
- sd, B2E10, 640, 1m18s, nan
- sd, B1E2, 640, , loss
- sd, B3E2, 640, , nan
- sd, B4E2, 640, , nan
- sd, B5E2, 640, , nan
- sd, B6E2, 640, , nan
- sm, b1e1, 1280, 41s, loss
- sm, b2e1, 1280, 41s, nan
- sm, b3e1, 1280, 41s, nan
- Só B1 funciona no modelo M, independente de 640 ou 1280.
Modelo L
- sd, B2E10, 640, 2m51s, loss
- sd, B3E5, 640, 1m12s, loss
- sd, B3E10, 640, 2m10s, loss
- sd, B4E10, 640, 1m47s, nan, class 
- sd, B5E2, 640, 1m47s, nan, class
- sd, b3e5, 1280, 3m47, nan
- sd, b2e1, 1280, , nan
- sd, b2e2, 1280, 1m45, nan, com classes ao fim
- sd, b1e1, 1280, 54s, loss
- Com 1280, L só funciona no b1
- L funciona em b3 com 640, e em b1 com 1280.

Assim: vou tentar treinar com o maior dataset o modelo L e M, com b1.

In [ ]:
!yolo task=detect mode=train model=D:/Programação/CV/football_analysis/yolov8l.pt data=D:/Programação/CV/football_analysis/training/datasets/football-players-detection-12/data.yaml batch=1 epochs=20 imgsz=1280 plots=True

# Treinos pesados
Treinando com o fd o modelo L e M, com b1.
- L, fd, b1e1, 1280, 21m12
- M, fd, b1e1, 1280, 13m8

Checando os mAP que mostra no final do treino, parece que todos os do modelo M ficaram números maiores (veja em D>prog>cv>benchmark). O que isso significa?

vou reinserir o plots=True, que gera gráficos quando do treinamento. Pra tentar ver oq descubro depois.

- L, fd, b1e20, 1280, 395m (6h30)

Depois de treinar, podemos fazer o benchmark de nossos novos modelos e checar sua capacidade.

"mAP, mean Average Precision, across all classes and different IOU thresholds. Its a number for the overall performance of the object detector."

to salvando os resultados dos modelos em D>prog>cv>benchmark

In [ ]:
!yolo val model=D:\Programação\CV\football_analysis\training\runs\detect\2.lfdb1e201280\weights\best.pt data=D:\Programação\CV\football_analysis\training\datasets\football-players-detection-12\data.yaml imgsz=1280